In [337]:
import numpy as np
import pandas as pd
import matplotlib as plt
# pd.options.display.max_rows = 4000
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', None)

In [453]:
date_cols = ['DOB','dateReceivedOriginal','dateReceivedCurrent','latestReleaseDate','paroleHearingDate','maxExpirationDateParole','postReleaseMaxExpiration','paroleBoardDischargeDate']
date_cols_unpure = ['earliestRelaseDate','paroleEligibilityDate','conditionalReleaseDate','maxExpirationDate']
transition_table_cols = ['custodyStatus','dateReceivedOriginal','latestReleaseDate','earliestReleaseDate','conditionalReleaseDate','minSentence','maxSentence','crime1','class1','crime2','class2','crime3','class3','crime4','class4']
THIRTY_YRS = 10950

In [434]:
df = pd.read_csv("/Users/jpouls/recidiviz/nyrecidiviz/inmates2018.csv",index_col=0,parse_dates=date_cols,na_filter=False)
# df = pd.read_csv("/Users/jpouls/recidiviz/nyrecidiviz/inmates2018.csv",index_col=0,na_filter=False)

# dfs = [pd.read_csv("/Users/jazz/proj/doc-scraper/ny_inmate_data/inmates"+str(year)+".csv",index_col=0,parse_dates=date_cols,na_filter=False) for year in range(2015,2020)]
df.head()

,name,sex,DOB,race,custodyStatus,housing,dateReceivedOriginal,dateReceivedCurrent,admissionType,county,latestReleaseDate,latestReleaseType,minSentence,maxSentence,earliestReleaseDate,earliestReleaseType,paroleHearingDate,paroleHearingType,paroleEligibilityDate,conditionalReleaseDate,maxExpirationDate,maxExpirationDateParole,postReleaseMaxExpiration,paroleBoardDischargeDate,crime1,class1,crime2,class2,crime3,class3,crime4,class4
DIN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18A0001,KENNETH EDWARDS,MALE,1992-06-21,BLACK,RELEASED,DOWNSTATE,2018-01-02,2020-06-30,,NASSAU,2020-07-16,PAROLE DIV OF PAROLE,3.84,4.50,,,NaT,FULL MAXIMUM,02/06/2020,NONE,06/16/2025,NaT,2025-06-16,NaT,ATT ASSAULT 2ND,E,ATT CRIM POSS WEAP 2ND,D,BURGLARY 3RD,D,,
18A0002,MICHAEL PERRONE,MALE,1966-03-23,WHITE,RELEASED,COLLINS,2018-01-02,2018-01-02,,NASSAU,2018-12-06,MERIT RELEASE TO PAROLE,1.50,4.73,,,2019-04-01,MERIT APPR OPEN DATE/6 MO AFT MERT APPEAR,02/08/2019,10/04/2020,05/02/2022,NaT,NaT,NaT,ATT ROBBERY 3RD,E,,,,,,
18A0003,JOHN BETANCOURT,MALE,1988-06-14,HISPANIC,RELEASED,ALTONA,2018-01-02,2018-01-02,,NASSAU,2018-09-11,MERIT RELEASE TO PAROLE,1.50,3.00,,,2019-01-01,MERIT APPR OPEN DATE/6 MO AFT MERT APPEAR,08/26/2018,02/26/2019,02/26/2020,NaT,2020-03-11,NaT,CRIM POSS MARIHUANA 3RD,E,CRIM POSS FIREARM,E,,,,
18A0004,CHARLES GRANDBERRY,MALE,1965-04-14,BLACK,RELEASED,RIVERVIEW,2018-01-02,2019-11-15,,NASSAU,2020-03-16,PAROLE DIV OF PAROLE,1.50,3.00,,,2020-01-01,PAROLE VIOLATOR ASSESSED EXPIRATION,11/30/2018,NONE,06/18/2020,NaT,NaT,NaT,CRIM POSS STOLEN PROP 4TH,E,,,,,,
18A0005,DUWAN GROOVER,MALE,1980-01-04,BLACK,RELEASED,DOWNSTATE,2018-01-02,2019-11-01,,NASSAU,2019-11-18,PAROLE DIV OF PAROLE,2.00,4.00,,,2019-11-01,PAROLE VIOLATOR ASSESSED EXPIRATION,01/27/2018,NONE,05/22/2020,NaT,NaT,NaT,ROBBERY 3RD,D,CRIM CONTEMPT 1ST,E,,,,


In [340]:
# ignore reentries
df = df[df.dateReceivedOriginal == df.dateReceivedCurrent]
# ignore no crime records
df = df[df.crime1 != 'NO CRIME RECORD AVAIL']

# trim to essential columns
dftrim = df[transition_table_cols]

In [368]:
# set conditionalReleaseDate as earliestReleaseDate if 'NONE'
b = dftrim.copy()
mask = b['conditionalReleaseDate'] == 'NONE'
b.loc[mask,'conditionalReleaseDate'] = b['earliestReleaseDate']

In [458]:
# set conditionalReleaseDate as DATE_FAR_IN_FUTURE if 'LIFE'
c = b.copy()
mask = c['conditionalReleaseDate'] == 'LIFE'
c.loc[mask, 'conditionalReleaseDate'] = '2/22/2222'

# set enddate, with priority to latestReleaseDate, conditionalReleaseDate otherwise
c['enddate'] = c.latestReleaseDate.combine_first(c.conditionalReleaseDate)

# set LOS as timedelta between enddate and entrydate
c['LOS'] = c.enddate - c.dateReceivedOriginal

#remove records with erroneous negative LOS
c = c[c.LOS.dt.days > 0] 

In [459]:
# get most serious crime by class
def getCrime(df):
#     cls = df[['class1','class2','class3','class4']].to_numpy()
#     index = min(range(4), key=lambda i: cls[i] if cls[i] else 'Z')
#     df['crime'] = df['crime'+str(index+1)]
#     df['crime_class'] = cls[index]
#     return df
    
    df['crime'], df['crime_class'] = df['crime1'],df['class1']
    for i in [2,3,4]:
        cl = df['class'+str(i)]
        if not cl:
            return df
        if cl < df['crime_class']:
            df['crime'], df['crime_class'] = df['crime'+str(i)], cl
    return df

c = c.apply(getCrime,axis=1)
c['crime'] = c.crime + '|' + c.crime_class
c = c[['dateReceivedOriginal','LOS','crime']]
c['LOS'] = c.LOS.dt.days

dateReceivedOriginal    datetime64[ns]
LOS                              int64
crime                           object
dtype: object

In [509]:
d = c.copy()
mask = d['LOS'] > THIRTY_YRS
d.loc[mask,'LOS'] = THIRTY_YRS

In [555]:
transition_df_by_crime = []

d=d[d.crime=="BURGLARY 1ST|B"]
for crime in d.crime.unique()[:10]:
    by_crime = d[d.crime == crime]
    N = len(by_crime.index)
    LOS_count = d[d.crime == crime].groupby('LOS').count()
#     print(LOS_count)
    minus = lambda x: N-x
    # np.cumsum gets # of inmates released, minus + LOS_count gets # of inmates remaining
    LOS_count['n_left'] = minus(np.cumsum(LOS_count).crime)+LOS_count.crime
    LOS_count['transition'] = LOS_count.crime/(LOS_count.n_left)
    LOS_count['CRIME'] = crime
#     LOS_count['LOSi'] = LOS_count.index
    transition_df_by_crime.append(LOS_count[['transition','CRIME']])
#     print(LOS_count[['LOSi','transition','CRIME']])
#     print(LOS_count.tail())

transitions = pd.concat(transition_df_by_crime)
transitions.to_csv('/Users/jpouls/recidiviz/nyrecidiviz/mm_preprocessing/transitions2018.csv')
    
    
#         numLOS['cum'] = cum.crime
#     numLOS['cum_left'] = minus(cum.crime)
#     numLOS['out_of_this_many'] = numLOS.cum_left+nu/mLOS.crime
#     numLOS['transition'] = numLOS.crime/(numLOS.cum_left+numLOS.crime)

#     print(numLOS)
#     print(numLOS.index.to_numpy(),len(numLOS.index.to_numpy()))
#     print(numLOS.to_numpy(),len(numLOS.to_numpy()))
#     print(np.cumsum(numLOS))#,len(np.cumsum(numLOS.to_numpy())))
#     print(square(np.cumsum(numLOS.to_numpy())))


<class 'pandas.core.frame.DataFrame'>


In [ ]:
d = c.copy()
for cr in d.crime.unique()[0:5]:
    e = d[d.crime == cr]
    count, bins_count = np.histogram(e.dt.days, bins=32)
    pdf = count / sum(count)
    cdf = np.cumsum(pdf) 
    print(cdf)
    los.hist(bins=32)
    

In [259]:
los = by_class['LOS'].count()
print(len(los.index))
print(max(los))
count, bins_count = np.histogram(los, bins=32)
pdf = count / sum(count)
los[los==""]
# los.sort()

6
3559


Series([], Name: LOS, dtype: int64)

In [ ]:
# OLD
# diffdates[~diffdates.paroleHearingType.str.contains("VIOLA")]
# merg = pd.merge(diffdates, parole_viol, how='inner', on=['DIN'])
# c['crime_short'] = c.crime.str.extract(r'(.*?\d).*')[0]

# s = d.cc.unique()
# s = [str(x) for x in s]
# s.sort()

# bins = np.arange(0.0, 36500.0, 365.0/12)
# bins = np.append(bins,max(d['LOS'].dt.days))
# count, bins_count = np.histogram(d['LOS'].dt.days, bins=bins)
# pdf = count / sum(count)
# cdf = np.cumsum(pdf) 
# d['LOS'].dt.days.hist(bins=bins)
# los.hist(bins=32)